In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Part 1
data = pd.read_csv('Instagram-Reach.csv')

# Convert to DateTime
data['Date'] = pd.to_datetime(data['Date'])

# Get Days of week and group by week
data['Day'] = data['Date'].dt.day_name()
dataDay = data.groupby('Day')
d30 = data[:30] # 30 days data

In [ ]:
# Data Info
data.head() # Checks for first 5 columns
data.isnull().sum()
data.describe()
data.info()

In [20]:
# Line Plot
data.plot('Date', 'Instagram reach', title='Instagram Reach over Time')

In [22]:
# Bar plot
data.plot.bar('Date', 'Instagram reach', title='Instagram Reach over Time')

In [24]:
# Box Plot
data.plot.box(title='Instagram Reach over Time')

In [ ]:
# Mean for Weekly Grouped Data
print("Mean: ")
print(data.groupby('Day').mean())
print("\nMedian: ")
print(data.groupby('Day').median())
print("\nStandard Deviation: ")
print(data.groupby('Day').std())


In [ ]:
# Bar Plot per Week Day
dataDay.plot.bar(x='Day', y='Instagram reach', title='Instagram Reach over Time')

In [ ]:
# Bar Plot per Weekday combined for 30 days
sns.catplot(
    x="Date",       # x variable name
    y="Instagram reach",       # y variable name
    hue="Day",  # group variable name
    data=d30,     # dataframe to plot
    kind="bar",
)